In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder, GTSRB
from torchvision.transforms import transforms as T
import torch.nn.functional as F
from pytorch_lightning.loggers import WandbLogger

from PIL import Image
from pytorch_lightning import LightningModule, Trainer
from sklearn.model_selection import train_test_split
from os.path import join
import kornia as K
import pandas as pd
import glob

In [2]:
set = GTSRB(root='./data/GTSRB/',download=True) # 43 classes

In [3]:
PATH = '/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs'
jpg = glob.glob(PATH + '/*/*.jpg')
jpg[:5]

['/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs/1.2/28.06.2022 full____NO20220628-175643-000064F_1_1652.jpg',
 '/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs/1.2/3____Screen (214)_375.jpg',
 '/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs/1.2/28.06.2022 full____NO20220628-135134-000021F_8_253.jpg',
 '/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs/1.2/27062022____NO20220627-161526-000096F_17_2182.jpg',
 '/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs/1.2/27062022____NO20220627-130026-000031F_25_599.jpg']

In [4]:
def parse_data(list_path):
    return [
        [
            item,
            item.split("/")[-2]
        ]
        for item in list_path
    ]


def class2label(item):
    if "noise" in item:
        return "noise"
    if len(set(item.split("/"))) == len(item.split("/")):
        ret = item.split("/")[1] if len(item.split("/")) > 1 else item
    else:
        ret = item.split("/")[-1]
    return ret


def build_DataFrame(data):
    df = pd.DataFrame(data)
    df = df.rename({0: "dir", 1: "class"}, axis=1)
    # df.drop_duplicates(subset=["name"], inplace=True)
    # df["label"] = df["class"].apply(class2label)
    # df["dir"] = df.apply(lambda x: join(PATH, x["class"], x["path"], x["name"]), axis=1)
    # df = df[["dir", "label"]]
    # df.reset_index(drop=True, inplace=True)
    return df

def build_label(data):
    # uniq = data.iloc[:, 1:].columns
    # len_uniq = len(uniq)
    # range_uniq = range(len_uniq)
    # d_ = dict(zip(range_uniq, uniq))
    # weights = data.iloc[:, 1:].values.sum() / data.iloc[:, 1:].sum()
    # return d_, list(weights)
    uniq = data['class'].unique()
    len_uniq = len(uniq)
    range_uniq = range(len_uniq)
    d_ = dict(zip(range_uniq, uniq))
    d = dict(zip(uniq, range_uniq))
    # weights = list(data.iloc[:, 1:].values.sum() / data.iloc[:, 1:].sum()
    return d_, d

def image_to_tensor(path):
    tensor = Image.open(path)
    return T.ToTensor()(tensor).float()

In [5]:
data = parse_data(jpg)
df = build_DataFrame(data)
df.head()

,dir,class
0,/root/workspace/work/Digital-Tashkent/Signs/tr...,1.2
1,/root/workspace/work/Digital-Tashkent/Signs/tr...,1.2
2,/root/workspace/work/Digital-Tashkent/Signs/tr...,1.2
3,/root/workspace/work/Digital-Tashkent/Signs/tr...,1.2
4,/root/workspace/work/Digital-Tashkent/Signs/tr...,1.2


In [6]:
num2label, label2num = build_label(df)
sampledf = df.sample(weights = 1./df.groupby('class')['class'].transform('count'))
sampledf

,dir,class
3520,/root/workspace/work/Digital-Tashkent/Signs/tr...,1.20


In [7]:
df.sample(weights = 1./df.groupby('class')['class'].transform('count'))['dir'].values[0]

'/root/workspace/work/Digital-Tashkent/Signs/traffic-sign-recognition/classification/data/signs/1.20/27062022____NO20220627-134826-000047F_1_987.jpg'

In [8]:
df.groupby('class')['class'].transform('count')

0         18
1         18
2         18
3         18
4         18
        ... 
12377    308
12378    308
12379    308
12380      2
12381      2
Name: class, Length: 12382, dtype: int64

In [9]:
# df_1 = pd.get_dummies(df, columns=['class'])
# num2label, weights = build_label(df)


In [10]:
class PreProcess(nn.Module):
    """Module to perform pre-process using Kornia on torch tensors."""

    def __init__(self, keepdim=True) -> None:
        super().__init__()
        self.preproc = nn.Sequential(
            K.augmentation.PadTo((50,50), keepdim=keepdim),
            K.augmentation.RandomAffine(
                degrees=20,
                translate=(0.1, 0.1),
                scale=(0.7, 1.4),
                p=0.8,
                keepdim=keepdim
            )
        )
        self.resize1= K.augmentation.LongestMaxSize(50)

    @torch.no_grad()  # disable gradients for effiency
    def forward(self, x_out: torch.Tensor) -> torch.Tensor:
        x_out = self.resize1(x_out.to(torch.float))
        x_out = self.preproc(x_out)

        # return x_out.to(torch.float16)
        return x_out[0]


In [11]:
class ImageDataset(Dataset):
    """
    Description:
        Torch DataSet for image loading and preprocessing.
        :data: pandas.DataFrame
        :root: str
        :transforms: nn.Sequential
        :label2num: dict
    """

    def __init__(
        self,
        data: pd.DataFrame,
        transforms: nn.Sequential,
        label2num=None,
        root: str = "",
        
    ):
        self.data = data
        self.transforms = transforms
        self.label2num = label2num
        self.root = root

    def parse_data(self, data):
        path = data["dir"].values[0]
        img, label = image_to_tensor(path), data["class"].values[0]
        return img, label

    @torch.no_grad()
    def __getitem__(self, idx):
        data = self.data.sample(weights = 1./self.data.groupby('class')['class'].transform('count'))
        img, label = self.parse_data(data)
        if self.transforms:
            img = self.transforms(img)
        # print(label, int(self.label2num[str(label)]))
        # torch.tensor(int(self.label2num[str(label)]), dtype=torch.float16)
        return img.to(torch.float16), torch.tensor(self.label2num[label], dtype=int)

    def __len__(self):
        return len(self.data)

In [12]:
ds = ImageDataset(df, PreProcess(), label2num)
# T.ToPILImage()(ds[0][0][0])

In [13]:
ds[0]

(tensor([[[0.0928, 0.0778, 0.0770,  ..., 0.0000, 0.0000, 0.0000],
          [0.0862, 0.0779, 0.0905,  ..., 0.0000, 0.0000, 0.0000],
          [0.0793, 0.0772, 0.1073,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.1300, 0.1741, 0.2124,  ..., 0.0000, 0.0000, 0.0000],
          [0.1398, 0.1752, 0.2057,  ..., 0.0000, 0.0000, 0.0000],
          [0.1453, 0.1608, 0.2001,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0870, 0.0789, 0.0817,  ..., 0.0000, 0.0000, 0.0000],
          [0.0750, 0.0770, 0.0933,  ..., 0.0000, 0.0000, 0.0000],
          [0.0635, 0.0740, 0.1073,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.2181, 0.2507, 0.3000,  ..., 0.0000, 0.0000, 0.0000],
          [0.2211, 0.2494, 0.2869,  ..., 0.0000, 0.0000, 0.0000],
          [0.2437, 0.2491, 0.2839,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.1699, 0.1704, 0.1588,  ..., 0.0000, 0.0000, 0.0000],
          [0.1637, 0.1682, 0.1643,  ..., 0.0000, 0.0000, 0.0000],
          [0.1580, 0.1661, 0.1749,  ...,

In [14]:
ds[0][1]

tensor(30)

In [15]:
loader = DataLoader(ds, 2000, num_workers=24)

In [16]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    dirpath=f"../lightning_logs/cls/", save_top_k=2, monitor="val_epoch_total_step", mode='max')


In [17]:
wandb_logger = WandbLogger(
    project='cls_signs',
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: timasaviin (tsu). Use `wandb login --relogin` to force relogin


In [18]:
from torchvision import models as M

In [19]:
def get_resnet(NUM_CLASSES):
    model = M.resnet18(weights=M.ResNet18_Weights.DEFAULT)
    in_feat = model.fc.in_features
    model.fc = nn.Linear(in_features=in_feat, out_features=NUM_CLASSES)
    return model

In [20]:
model = get_resnet(len(num2label))

In [21]:
class FasterRCNN(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = model
        self.lr = 1e-4
        self.sigmoid = nn.Sigmoid()
        # self.cross_entropy = nn.BCEWithLogitsLoss(
        #     pos_weight=self.get_pos_weight(weights), 
        #     reduction="none"
        # )
        self.cross_entropy = nn.CrossEntropyLoss(reduction='sum')
        self.save_hyperparameters()


    def get_pos_weight(self, weights):
        pos_weight = torch.tensor(weights) if type(weights) == list else None
        return pos_weight
        
    def forward(self, imgs):
        return self.model(imgs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=0.1)
        scheduler = {'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer, mode='min', factor=0.3, patience=3), 'monitor': 'train_loss', }
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
        }

    def training_step(self, batch, batch_idx):
        images, targets = batch
        logits = self.forward(images)
        loss = self.cross_entropy(logits, targets)
        #################################
        # pt = torch.exp(-loss)
        # loss = (2 * (1-pt)**1 * loss).sum()
        self.log('train_loss', loss.detach(), on_step=True)
        return loss


In [22]:
trainer = Trainer(accelerator='gpu', devices=1,
                  max_epochs=1000,
                  precision=16,
                  log_every_n_steps=5,
                  logger=wandb_logger,
                  )

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
model = FasterRCNN()
trainer.fit(model,
            loader,
            )

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | ResNet           | 11.2 M
1 | sigmoid       | Sigmoid          | 0     
2 | cross_entropy | CrossEntropyLoss | 0     
---------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
22.467    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/root/miniconda3/envs/sign-rec/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# label2num = {v:k for k, v in ds.class_to_idx.items()}

AttributeError: 'ImageDataset' object has no attribute 'class_to_idx'

In [80]:
i=8790
img = ds[i]
num2label[int(img[1])], int(img[1])


('3.4', 95)

In [81]:
num2label[model(img[0].float().cuda().unsqueeze(0)).argmax().item()], model(img[0].float().cuda().unsqueeze(0)).argmax().item()

('5.16.2', 59)

In [ ]:
num2label

{0: '1.2',
 1: '1.13',
 2: '3.26',
 3: '7.13',
 4: '4.2.2',
 5: '5.6',
 6: '7.3.1',
 7: '4.5',
 8: '6.3',
 9: '1.11.1',
 10: '3.12',
 11: '7.2.3',
 12: '5.17.2',
 13: '3.13',
 14: '4.1.1',
 15: '1.31.2',
 16: '5.19.1',
 17: '3.24',
 18: '4.11.1',
 19: '1.4.3',
 20: '5.41',
 21: '1.11.2',
 22: '5.15',
 23: '3.18.1',
 24: '5.1.11',
 25: '5.8.3',
 26: '5.19.3',
 27: '3.19',
 28: '5.5',
 29: '7.4.1',
 30: '4.1.4',
 31: '1.4.1',
 32: '1.3.1',
 33: '5.38',
 34: '2.5',
 35: '3.5',
 36: '7.21',
 37: '5.42',
 38: '3.10',
 39: '1.1',
 40: '4.6.3',
 41: '6.4',
 42: '7.2.4',
 43: '5.12',
 44: '5.43',
 45: '7.22.3',
 46: '2.4.',
 47: '3.28',
 48: '4.6.1',
 49: '5.33',
 50: '1.4.2',
 51: '1.30',
 52: '1.4.4',
 53: '4.3',
 54: '1.31',
 55: '5.8.5',
 56: '4.1.2',
 57: '1.20',
 58: '1.12.2',
 59: '5.16.2',
 60: '4.6',
 61: '2.3.2',
 62: '1.7',
 63: '1.21',
 64: '7.22.2',
 65: '3.20',
 66: '1.4.6',
 67: '1.31.1',
 68: '5.29.1',
 69: '7.6.1',
 70: '5.7.1',
 71: '1.3.2',
 72: '5.8.1',
 73: '3.25',
 74: '3